# Lab 3 - Enter the Model Context Protocol!

In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio

import os
load_dotenv(override=True)
MODEL_NAME = "gpt-4.1-mini"

In [4]:
import sys
print("Platform:", sys.platform)
print("WSL check:", "microsoft" in open("/proc/version").read().lower() if sys.platform == "linux" else "Not Linux")

Platform: win32
WSL check: Not Linux


In [5]:
# An MCP Server is defined by Parameters

fetch_params = {"command": "npx", "args": ["-y", "@playwright/mcp@latest", "--headless=false"]}
async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
    tools = await fetch.session.list_tools()
print("Fetch tools:", tools.tools)

Error initializing MCP server: fileno


UnsupportedOperation: fileno

In [ ]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
    file_tools = await filesystem.session.list_tools()

file_tools.tools

In [ ]:
instructions = "You use your tools to navigate the web and write summaries to a file"

input = "Bring up a browser, visit 3 different news sites, click on at least 1 story on each site, and write a summary of the news in markdown to a file news.md in the sandbox directory"

In [ ]:
with trace("News"):
    async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
        async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
            agent = Agent(name="News", instructions=instructions, model=MODEL_NAME, mcp_servers=[fetch, filesystem])
            result = await Runner.run(agent, input, max_steps=20)
            print(result.final_output)

# And now.. a hands on moment

### Everyone write a version of this that uses the MCP Server from Playwright instead of from Fetch

Using this MCP Server from Microsoft:

https://mcp.so/server/playwright-mcp/microsoft

But NOT running it in headless mode, so you get to see the browser

### And check out the traces

https://platform.openai.com/traces